In [27]:
import gtsam

In [2]:
#### FILTER OBJECT
class GraphTracker():
    def __init__(self):
        
        # Member variables
        self.init = False
        self.detections = []
        self.tracks = []
        self.graph = gtsam.NonlinearFactorGraph()
        self.trk_delete_thresh = 0.25
        self.trk_delete_age = 10
        self.trk_create_thresh = 0.75

    # Member functions
    def handle_detections(self,det_array_msg):
        self.detections = det_array_msg.boxes
        
        # TODO - Compute detection-track correspondences
        
        # TODO - Assign detections to track variables
        
        # TODO - create tracks as needed
        
        # Delete tracks as needed
        self.delete_tracks()
        
    def delete_tracks(self):
        self.tracks = [track for track in self.tracks if track.prob < self.trk_delete_thresh]
        self.tracks = [track for track in self.tracks if track.age > self.trk_delete_age]
        
    # birth
    # compute assignment
    # add factors
    # optimize

In [5]:
#### DETECTION OBJECT
class Detection():
    def __init__(self, det_msg):
        self.px = det_msg.pose.position.x
        self.py = det_msg.pose.position.y
        self.pz = det_msg.pose.position.z
        self.age = 0
        self.prob = det_msg.value
        self.class = det_msg.label
        # Can be empty/null/contain partial info
        # class conf % - float
        # ??? BB dimensions

#### TRACK OBJECT
class Track():
    def __init__(self, det):
        self.px = det.px
        self.py = det.py
        self.pz = det.pz
        self.vx = 0
        self.vy = 0
        self.vz = 0
        self.prob = 
        self.age = det.age
        # heading in world frame
        # self.class = categorical distribution!
        # ??? class conf %

In [ ]:
#### HELPER FUNCTIONS
# Similarity metric (detection, track)

# Form similarity matrix (detections, tracks) -> return similarity matrix
# - call similarity metric

# Hungarian/Munkres algo (similarity matrix) -> return assignments (T_match, D_match, T_unmatch (missed/occ), D_unmatch (false, new obj, extra detection)


In [3]:
import numpy as np
from scipy.optimize import linear_sum_assignment

# Create a cost matrix
cost_matrix = np.array([
    [4, 1, 3],
    [2, 0, 5],
    [3, 2, 2],
    [0, 0, 0]
])

# Solve the assignment problem
row_indices, col_indices = linear_sum_assignment(cost_matrix)

# Extract the optimal assignment
assignment = [(row, col) for row, col in zip(row_indices, col_indices)]

print("Optimal Assignment:")
for row, col in assignment:
    print(f"Resource {row} assigned to Task {col}")


Optimal Assignment:
Resource 1 assigned to Task 1
Resource 2 assigned to Task 2
Resource 3 assigned to Task 0


In [25]:
def cost(det,track):
    return np.abs(det-track)

def compute_cost_matrix(detects, tracks):
    cost_matrix = np.zeros((len(detects),len(tracks)))
    for ii,det in enumerate(detects):
        for jj,trk in enumerate(tracks):
            cost_matrix[ii,jj] = cost(det,trk)
    return cost_matrix  

tracks = [1, 2, 3]
det = [3.1, 3.01, 0.9, 4.4, 2.1]

cost_matrix = compute_cost_matrix(det,tracks)

# Solve the assignment problem
row_indices, col_indices = linear_sum_assignment(cost_matrix)

# Extract the optimal assignment
assignment = [(row, col) for row, col in zip(row_indices, col_indices)]

In [26]:
print(cost_matrix)

print("Assign this detection")
print(row_indices)
print("To this track")
print(col_indices)
print(assignment)

[[2.1  1.1  0.1 ]
 [2.01 1.01 0.01]
 [0.1  1.1  2.1 ]
 [3.4  2.4  1.4 ]
 [1.1  0.1  0.9 ]]
Assign this detection
[1 2 4]
To this track
[2 0 1]
[(1, 2), (2, 0), (4, 1)]


In [21]:
import numpy as np
my_arr = np.array([[0.],[0.],[0.]])
my_arr2 = np.array([[1.],[2.],[3.]])


In [24]:
np.linalg.norm(my_arr - my_arr2)

3.7416573867739413

# Kalman filter testing

In [126]:
dir(kf)

['FromMeanAndStddev',
 'R',
 'S',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'augmentedInformation',
 'augmentedJacobian',
 'augmentedJacobianUnweighted',
 'clone',
 'cols',
 'covariance',
 'd',
 'deserialize',
 'eliminate',
 'empty',
 'equals',
 'error',
 'error_vector',
 'evaluate',
 'firstFrontalKey',
 'getA',
 'get_model',
 'getb',
 'information',
 'isConstrained',
 'jacobian',
 'jacobianUnweighted',
 'keys',
 'likelihood',
 'logNormalizationConstant',
 'logProbability',
 'mean',
 'negate',
 'print',
 'printKeys',
 'rows',
 'sample',
 'serialize',
 'setModel',
 'size',
 'solve',
 'solveOtherRHS',
 'solveTransposeInPlace',
 'transposeMultiplyAdd',
 'unweig

In [159]:
# Initialization
pos = np.array([[1],[2],[3]])
vel = np.array([[4],[5],[6]])

x0 = np.vstack((pos,vel))
P0 = np.diag([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

kf = gtsam.KalmanFilter(6)
state = kf.init(x0, P0)
print(state.mean()[0:3])

[1. 2. 3.]


In [154]:
# prediction
dt = .03

F = np.array([[1., 0, 0, dt, 0, 0],
               [0, 1., 0, 0, dt, 0],
               [0, 0, 1., 0, 0, dt],
               [0, 0, 0, 1., 0, 0],
               [0, 0, 0, 0, 1., 0],
               [0, 0, 0, 0, 0, 1.]])

Q = gtsam.noiseModel.Diagonal.Sigmas([1,2,3,4,5,6])
print(Q)

kf.predict(state,F,np.zeros((6,6)),np.zeros((6,1)),Q)

diagonal sigmas [1; 2; 3; 4; 5; 6];



GaussianDensity: density on [1] 
mean: [1.12; 2.15; 3.18; 4; 5; 6];
covariance: [
	1.50045, 0, 0, 0.015, 0, 0;
	0, 4.50045, 0, 0, 0.015, 0;
	0, 0, 9.50045, 0, 0, 0.015;
	0.015, 0, 0, 16.5, 0, 0;
	0, 0.015, 0, 0, 25.5, 0;
	0, 0, 0.015, 0, 0, 36.5
]
Noise model: unit (6) 

In [158]:
print(F)
F[2,2] = 0.5
print(F)

[[1.   0.   0.   0.03 0.   0.  ]
 [0.   1.   0.   0.   0.03 0.  ]
 [0.   0.   1.   0.   0.   0.03]
 [0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   1.  ]]
[[1.   0.   0.   0.03 0.   0.  ]
 [0.   1.   0.   0.   0.03 0.  ]
 [0.   0.   0.5  0.   0.   0.03]
 [0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   1.  ]]


In [150]:
dir(gtsam)

['AHRSFactor',
 'AcceleratingScenario',
 'BatchFixedLagSmoother',
 'BearingFactor2D',
 'BearingFactor3D',
 'BearingFactorPose2',
 'BearingRange2D',
 'BearingRange3D',
 'BearingRangeFactor2D',
 'BearingRangeFactor3D',
 'BearingRangeFactorPose2',
 'BearingRangeFactorPose3',
 'BearingRangePose2',
 'BearingRangePose3',
 'BetweenFactorConstantBias',
 'BetweenFactorDouble',
 'BetweenFactorPoint2',
 'BetweenFactorPoint3',
 'BetweenFactorPose2',
 'BetweenFactorPose2s',
 'BetweenFactorPose3',
 'BetweenFactorPose3s',
 'BetweenFactorRot2',
 'BetweenFactorRot3',
 'BetweenFactorSO3',
 'BetweenFactorSO4',
 'BetweenFactorVector',
 'BinaryMeasurementPoint3',
 'BinaryMeasurementRot3',
 'BinaryMeasurementUnit3',
 'BinaryMeasurementsPoint3',
 'BinaryMeasurementsRot3',
 'BinaryMeasurementsUnit3',
 'BlockJacobiPreconditionerParameters',
 'Cal3Bundler',
 'Cal3DS2',
 'Cal3DS2_Base',
 'Cal3Fisheye',
 'Cal3Unified',
 'Cal3_S2',
 'Cal3_S2Stereo',
 'CalibratedCamera',
 'CameraSetCal3Bundler',
 'CameraSetCal3DS2'